<a href="https://colab.research.google.com/github/HofstraDoboli/TextMining_F22/blob/main/probabilistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
list_doc =[ "Information retrieval course will cover algorithms used in search engines for finding relevant documents or information related to a query. Topics include: natural language processing for extracting relevant terms out of text data, vector space a methods for computing similarity between documents, text classification, and clustering. These techniques are commonly used in applications such as: automatic extraction of summaries out of a long text, extract novel information in a stream of data.",  

"NLP algorithms are typically based on machine learning algorithms. Instead of hand-coding large sets of rules, NLP can rely on machine learning to automatically learn these rules by analyzing a set of examples (i.e. a large corpus, like a book, down to a collection of sentences), and making a statical inference. In general, the more data analyzed, the more accurate the model will be.",  

"The Denver Broncos made sure Brandon McManus will be their kicker for the long haul on Monday.  General manager John Elway announced the team and the kicker agreed on a contract extension. NFL Network Insider Ian Rapoport reported, per a source, it's a three-year extension worth $11.254 million with $6 million of it guaranteed. McManus is now the NFL's fourth highest paid kicker.", 

 "Equifax, one of the three major credit reporting agencies, handles the data of 820 million consumers and more than 91 million businesses worldwide. Between May and July of this year 143 million people in the U.S. may have had their names, Social Security numbers, birth dates, addresses and even driver's license numbers accessed. In addition, the hack compromised 209,000 people's credit card numbers and personal dispute details for another 182,000 people. What bad actors could do with that information is daunting. This data breach is more confusing than others -- like when Yahoo or Target were hacked, for example -- according to Joel Winston, a former deputy attorney general for New Jersey , whose current law practice focuses on consumer rights litigation, information privacy, and data protection law.", 
 
  """Why didn't she text me back yet? She doesn't like me anymore!" "There's no way I'm trying out for the team. I suck at basketball""It's not fair that I have a curfew! "Sound familiar? Parents of tweens and teens often shrug off such anxious and gloomy thinking as normal irritability and moodiness — because it is. Still, the beginning of a new school year, with all of the required adjustments, is a good time to consider just how closely the habit of negative, exaggerated "self-talk" can affect academic and social success, self-esteem and happiness. Psychological research shows that what we think can have a powerful influence on how we feel emotionally and physically, and on how we behave. Research also shows that our harmful thinking patterns can be changed."""]

In [2]:
import numpy as np
import spacy   # another tokenizer, lemmatizer (has --> be)
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes('parser', 'ner') 

['parser', 'ner']

In [9]:
# Step 1: text processing for one document - return lemmas
def nlp_processing(doc):  
    tokens = nlp(doc)
    
    #print(type(tokens))
    # eliminates stop words  and non alpha num and converts all to lower case
    terms = [token.lemma_.lower() for token in tokens if not token.is_stop and token.is_alpha] 
  
    return terms

# Step 2: extract a list of (token, doc_id) from all documents.
# input a list of documents
# output: a list of sorted (token, doc_id) tuples
def extract_token_doc_id(list_doc):
  all_tokens = []
  len_docs = [0]*len(list_doc)
  for ind_doc, doc in enumerate(list_doc):
    tokens = nlp_processing(doc)
    len_docs[ind_doc] = len(tokens)
    tokens_doc = [(token, ind_doc) for token in tokens]
    all_tokens.extend(tokens_doc)
  
  # sort by token name 
  all_tokens = sorted(all_tokens, key = lambda x:x[0])

  return all_tokens, len_docs

def counter(items):
  sort_items = sorted(items) # sorts tokens alphabetically
  count_items = {}
  for item in sort_items:
    if item in count_items.keys():
      count_items[item] += 1
    else:
      count_items[item] = 1
  
  # sort by the count, in reverse order
  sorted_count_list = sorted(count_items.items(), 
                            key = lambda x:x[1], reverse = True)
  sorted_count_dict = dict(sorted_count_list)
  return sorted_count_dict 


# Step 3: Extract terms (unique) and document frequency (count tokens)
# change this to account only once for a repeated term in the same document
# all_tokens list of tuples
def doc_freq(all_tokens):
  set_all_tokens = set(all_tokens) # remove duplicate token in the same document
  dict_doc_freq = {}
  for (token, doc) in set_all_tokens:
    if token in dict_doc_freq:
      dict_doc_freq[token] += 1
    else: 
      dict_doc_freq[token] = 1

  # sort by key (term)  
  tuples_doc_freq = sorted(dict_doc_freq.items(), key = lambda x: x[0])
  
  dict_doc_freq = {term:doc_freq for (term, doc_freq) in tuples_doc_freq}
  return dict_doc_freq

# Step 4: Extract term frequency of each term in each document it appears in
# dict_term_freq = {term: {doc1:tf1, doc2:tf2, ...}} # includes only docs that have 
# non-zero term frequency
def term_freq(all_tokens, dict_doc_freq):
  dict_term_freq = {term:{} for term in dict_doc_freq.keys()} # initialize dictionary with all unique terms
  for (token, doc) in all_tokens:
    if doc in dict_term_freq[token]:
      dict_term_freq[token][doc] += 1 
    else: # if doc is not a key in the dictionary 
      dict_term_freq[token][doc] = 1
  
  return dict_term_freq

In [11]:
# nlp processing on all docs, extract len_docs
list_token_doc, len_docs = extract_token_doc_id(list_doc)
print(list_token_doc[:5])
print(len_docs)

[('academic', 4), ('access', 3), ('accord', 3), ('accurate', 1), ('actor', 3)]
[45, 37, 38, 70, 59]


In [21]:
# compute the P(w|collection) = count of a word in the collection/sum length docs
len_collection = sum(len_docs)
all_tokens = [t for (t,d) in list_token_doc]
dict_collection = counter(all_tokens)
print(list(dict_collection.items())[:5])
print(list(dict_collection.items())[-5:])


[('information', 5), ('million', 5), ('datum', 4), ('text', 4), ('algorithm', 3)]
[('way', 1), ('winston', 1), ('worldwide', 1), ('worth', 1), ('yahoo', 1)]


In [20]:
dict_term_freq = term_freq(list_token_doc, dict_count)
print(list(dict_term_freq.items())[:5])

[('information', {0: 3, 3: 2}), ('million', {2: 2, 3: 3}), ('datum', {0: 2, 1: 1, 3: 1}), ('text', {0: 3, 4: 1}), ('algorithm', {0: 1, 1: 2})]


In [36]:
# jm similarity method
import numpy as np

def jm_similarity(query, len_docs, dict_freq_collection, dict_term_freq,
                  lam = 0.2):
  # process the query - > word and count in the query
  query_terms = nlp_processing(query)
  dict_freq_query = counter(query_terms)
  similarity = {}
  # find the set of docs with at least 1 query word
  len_all = sum(len_docs)
  words_query = dict_freq_query.keys()
  docs = [] 
  for w in words_query:
    docs.extend(dict_term_freq[w].keys())
  docs = set(docs)

  similarity = {d:0 for d in docs}
  # for each word in the query
  for w in dict_freq_query.keys():
  #   for each doc the word appears in
    if w not in dict_freq_collection: # if word does not appear in the collection skip over it
      continue
    # check the documents w appears in
    for d in docs:
      # compute P(w|d) = (1-lambda) * c(w,d)/len(d) + lambda*c(w,C)/sum(len)
      c_w_d = 0
      if d in dict_term_freq[w]:
        c_w_d = dict_term_freq[w][d]

      prob_w_d = (1-lam)* c_w_d/len_docs[d] + (lam*dict_freq_collection[w])/len_all

      # add to the similarity of (q,d) =  c(w,q) * log(P(w|d)
      similarity[d] += dict_freq_query[w] * np.log(prob_w_d)

  # sort the similarity by value 
  sorted_sim = sorted(similarity.items(), key = lambda x: x[1], reverse = True)
  return sorted_sim

In [37]:
ranked_doc = jm_similarity("information retrieval", len_docs, dict_collection, dict_term_freq, 0.2)
ranked_doc

[(0, -6.844209169607428), (3, -10.743516315689313)]